In [1]:
import sys

import numpy as np
from process_bigraph import pp

sys.path.insert(0, '..')

In [2]:
"""
Interpretation of Metrics
Low Inner Product & High MSE/RMSE:

Description: Simulators produce outputs that are different both in magnitude and direction.
Example: If inner_product_matrix[0, 2] is low and mse_matrix[0, 2] is high, copasi and amici are very different.
High Inner Product & Low MSE/RMSE:

Description: Simulators produce very similar and aligned outputs.
Example: If inner_product_matrix[0, 1] is high and mse_matrix[0, 1] is low, copasi and tellurium are very similar.
High Inner Product & High MSE/RMSE:

Description: Simulators produce outputs that are aligned in direction but have significant differences in magnitude.
Example: This scenario might indicate systematic biases in the simulators' outputs.
Low Inner Product & Low MSE/RMSE:

Description: Simulators produce similar outputs, but the alignment is not perfect.
Example: This might indicate outputs that are close but vary slightly in their direction.
"""

"\nInterpretation of Metrics\nLow Inner Product & High MSE/RMSE:\n\nDescription: Simulators produce outputs that are different both in magnitude and direction.\nExample: If inner_product_matrix[0, 2] is low and mse_matrix[0, 2] is high, copasi and amici are very different.\nHigh Inner Product & Low MSE/RMSE:\n\nDescription: Simulators produce very similar and aligned outputs.\nExample: If inner_product_matrix[0, 1] is high and mse_matrix[0, 1] is low, copasi and tellurium are very similar.\nHigh Inner Product & High MSE/RMSE:\n\nDescription: Simulators produce outputs that are aligned in direction but have significant differences in magnitude.\nExample: This scenario might indicate systematic biases in the simulators' outputs.\nLow Inner Product & Low MSE/RMSE:\n\nDescription: Simulators produce similar outputs, but the alignment is not perfect.\nExample: This might indicate outputs that are close but vary slightly in their direction.\n"

In [3]:
biomodel_id = 'BIOMD0000000630'
duration = 100

# sample time series outputs from three simulators
output_copasi = np.array([1.3, 2.4, 4.3, 5.6, 6.7])
output_tellurium = np.array([1.3, 2.4, 4.5, 5.7, 6.8])
output_amici = np.array([1.3, 2.4, 4.6, 5.8, 7.0])
interval_outputs = [output_copasi, output_tellurium, output_amici]
feature_names = ['param_a', 'param_b', 'param_c', 'param_d', 'param_e']

# sample time id for a random interval in the range of duration
time_id = np.random.randint(0, duration)

In [4]:
from biosimulator_processes.data_model.compare_data_model import ODEProcessIntervalComparison
from biosimulator_processes.verify.exec import generate_ode_process_interval_comparison_data, plot_ode_process_comparison

CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess registered successfully.
GetSbmlStep registered successfully.
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [5]:
# generate object with comparison dataframes for sample data
interval_comparison_data: ODEProcessIntervalComparison = generate_ode_process_interval_comparison_data(interval_outputs, time_id)

# show time id for sample interval
interval_comparison_data.time_id

28

In [6]:
# show mse data
interval_comparison_data.mse_data

,copasi,tellurium,amici
copasi,0.000,0.012,0.044
tellurium,0.012,0.000,0.012
amici,0.044,0.012,0.000


In [7]:
# show inner product data 
interval_comparison_data.inner_prod_data

,copasi,tellurium,amici
copasi,102.19,104.28,106.61
tellurium,104.28,106.43,108.81
amici,106.61,108.81,111.25


In [8]:
# show rmse data 
interval_comparison_data.rmse_data

,copasi,tellurium,amici
copasi,0.000000,0.109545,0.209762
tellurium,0.109545,0.000000,0.109545
amici,0.209762,0.109545,0.000000


In [9]:
# show outer product data
pp(interval_comparison_data.outer_prod_data)

{ ('amici', 'amici'): array([[ 1.69,  3.12,  5.98,  7.54,  9.1 ],
       [ 3.12,  5.76, 11.04, 13.92, 16.8 ],
       [ 5.98, 11.04, 21.16, 26.68, 32.2 ],
       [ 7.54, 13.92, 26.68, 33.64, 40.6 ],
       [ 9.1 , 16.8 , 32.2 , 40.6 , 49.  ]]),
  ('copasi', 'amici'): array([[ 1.69,  3.12,  5.98,  7.54,  9.1 ],
       [ 3.12,  5.76, 11.04, 13.92, 16.8 ],
       [ 5.59, 10.32, 19.78, 24.94, 30.1 ],
       [ 7.28, 13.44, 25.76, 32.48, 39.2 ],
       [ 8.71, 16.08, 30.82, 38.86, 46.9 ]]),
  ('copasi', 'copasi'): array([[ 1.69,  3.12,  5.59,  7.28,  8.71],
       [ 3.12,  5.76, 10.32, 13.44, 16.08],
       [ 5.59, 10.32, 18.49, 24.08, 28.81],
       [ 7.28, 13.44, 24.08, 31.36, 37.52],
       [ 8.71, 16.08, 28.81, 37.52, 44.89]]),
  ('copasi', 'tellurium'): array([[ 1.69,  3.12,  5.85,  7.41,  8.84],
       [ 3.12,  5.76, 10.8 , 13.68, 16.32],
       [ 5.59, 10.32, 19.35, 24.51, 29.24],
       [ 7.28, 13.44, 25.2 , 31.92, 38.08],
       [ 8.71, 16.08, 30.15, 38.19, 45.56]]),
  ('tellurium', 